In [10]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np


# 학습 파일 불러오기
# 
# 1. 라인 한줄한줄 불러옴
# 2. ;, $, 공백, 뺴고 [배치,문장] 리스트로 불러옴
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents


# 학습용 말뭉치 데이터를 불러옴
#corpus = read_file('train.txt')
corpus = read_file(r'C:\workspace\VScode_project\project2\chatbot_book_ex\book_ex\ch6\train.txt')
#print(type(corpus))
# print(corpus[0:1][0:1])

# 말뭉치 데이터에서 단어와 BIO 태그만 불러와 학습용 데이터셋 생성
sentences, tags = [], []
for t in corpus:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])

    sentences.append(sentence)
    tags.append(bio_tag)

print("샘플 크기 : \n", len(sentences))
print("0번째 샘플 문장 시퀀스 : \n", sentences[0])
print("0번째 샘플 bio 태그 : \n", tags[0])
print("샘플 문장 시퀀스 최대 길이 :", max(len(l) for l in sentences))
print("샘플 문장 시퀀스 평균 길이 :", (sum(map(len, sentences))/len(sentences)))


# 토크나이저 정의 및 피팅
# OOV: out of vocabulary, 사전에 포함되지 않은 단어
sent_tokenizer = preprocessing.text.Tokenizer(oov_token='OOV') # 첫 번째 인덱스에는 OOV 사용
sent_tokenizer.fit_on_texts(sentences)
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower= False 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)


# 단어 사전 및 태그 사전 크기
print("tag_tokenizer.word_index", tag_tokenizer.word_index)
vocab_size = len(sent_tokenizer.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1 # 패딩 처리
print("BIO 태그 사전 크기 :", tag_size)
print("단어 사전 크기 :", vocab_size)

# 학습용 단어 시퀀스 생성
x_train = sent_tokenizer.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)
print(x_train[0])
print(y_train[0])

# index to word / index to NER 정의
index_to_word = sent_tokenizer.index_word # 시퀀스 인덱스를 단어로 변환하기 위해 사용
index_to_ner = tag_tokenizer.index_word # 시퀀스 인덱스를 NER로 변환하기 위해 사용
print("before: index_to_ner", index_to_ner)
index_to_ner[0] = 'PAD' # 패딩 처리
print("after: index_to_ner", index_to_ner)

# 시퀀스 패딩 처리
max_len = 40  # 책에서는 평균길이 36보다 조금 큰 값으로 설정 --> 그러면 최대길이 168은 인식을 안하기로 한거?
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

# 학습 데이터와 테스트 데이터를 8:2 비율로 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=.2, random_state=0)
#x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=.2, random_state=0, stratify=y_train)

# 출력 데이터를 원-핫 인코딩
# value가 문자로 되어 있음
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)

print(type(y_test))


샘플 크기 : 
 3555
0번째 샘플 문장 시퀀스 : 
 ['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.']
0번째 샘플 bio 태그 : 
 ['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
샘플 문장 시퀀스 최대 길이 : 168
샘플 문장 시퀀스 평균 길이 : 34.03909985935302
tag_tokenizer.word_index {'O': 1, 'I': 2, 'B_OG': 3, 'B_PS': 4, 'B_DT': 5, 'B_LC': 6, 'B_TI': 7}
BIO 태그 사전 크기 : 8
단어 사전 크기 : 13834
[183, 11, 4276, 884, 162, 931, 402, 10, 2608, 7, 1516, 608, 145, 1361, 414, 4, 6347, 2, 8, 3]
[1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
before: index_to_ner {1: 'O', 2: 'I', 3: 'B_OG', 4: 'B_PS', 5: 'B_DT', 6: 'B_LC', 7: 'B_TI'}
after: index_to_ner {1: 'O', 2: 'I', 3: 'B_OG', 4: 'B_PS', 5: 'B_DT', 6: 'B_LC', 7: 'B_TI', 0: 'PAD'}
학습 샘플 시퀀스 형상 :  (2844, 40)
학습 샘플 레이블 형상 :  (2844, 40, 8)
테스트 샘플 시퀀스 형상 :  (711, 40)
테스트 샘플 레이블 형상 :  (711, 40, 8)
<class 'numpy.ndarray'>


In [16]:
# 모델 정의(Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam


# 모델 설정
# Embedding() --> tokenizer로 변형된 희소벡터를 밀집벡터로 변환
#   input_dim = vocab_size = 13834 --> 단어사전크기
#   input_length = max_len = 40 --> 임베드 레이어의 입력차원
#   output_dim = 30 --> 임베드 레이어의 출력 차원
#   mask_zero = True --> 설명 없음
# LSTM층 입력 차원을 정해주지 않았는데 자동으로 됨
# tag_size = BIO 사전 크기

model = Sequential()
# 입력층
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
# 히든레이어
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
# 출력층
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))

# 컴파일 & 학습
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10)
print("평가 결과 : ", model.evaluate(x_test, y_test)[1])


# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):
    result = []
    for sequence in sequences:
        temp = []
        for pred in sequence:
            pred_index = np.argmax(pred)
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))
        result.append(temp)
    return result


# 테스트 데이터셋의 NER 예측
y_predicted = model.predict(x_test) # (711, 40) => model => (711, 40, 8)
pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
test_tags = sequences_to_tag(y_test) # 실제 NER



Epoch 1/10
23/23 [==============================] - 13s 326ms/step - loss: 0.6995 - accuracy: 0.8359
Epoch 2/10
23/23 [==============================] - 7s 322ms/step - loss: 0.3154 - accuracy: 0.8999
Epoch 3/10
23/23 [==============================] - 8s 333ms/step - loss: 0.2006 - accuracy: 0.9319
Epoch 4/10
23/23 [==============================] - 8s 333ms/step - loss: 0.1366 - accuracy: 0.9567
Epoch 5/10
23/23 [==============================] - 8s 329ms/step - loss: 0.0900 - accuracy: 0.9730
Epoch 6/10
23/23 [==============================] - 8s 342ms/step - loss: 0.0676 - accuracy: 0.9796
Epoch 7/10
23/23 [==============================] - 8s 333ms/step - loss: 0.0521 - accuracy: 0.9838
Epoch 8/10
23/23 [==============================] - 8s 347ms/step - loss: 0.0444 - accuracy: 0.9860
Epoch 9/10
23/23 [==============================] - 8s 327ms/step - loss: 0.0354 - accuracy: 0.9889
Epoch 10/10
23/23 [==============================] - 1s 34ms/step - loss: 0.3088 - accuracy: 0.9400

ModuleNotFoundError: No module named 'seqeval'

In [17]:
!pip install seqeval

                                              0.0/43.6 kB ? eta -:--:--
     ---------------------------------------- 43.6/43.6 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16196 sha256=9a7ec4a6385b43ac27c0222bdace4aa981d278d896a16e6ea3b22f11f9c4e08b
  Stored in directory: c:\users\tjoeun\appdata\local\pip\cache\wheels\ad\5c\ba\05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [18]:
## F1 스코어 계산을 위해 사용
from seqeval.metrics import f1_score, classification_report
print(classification_report(test_tags, pred_tags))
print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))


# 새로운 유형의 문장 NER 예측
word_to_index = sent_tokenizer.word_index
new_sentence = '삼성전자 출시 스마트폰 오늘 애플 도전장 내밀다.'.split()
new_x = []
for w in new_sentence:
    try:
        new_x.append(word_to_index.get(w, 1))
    except KeyError:
        # 모르는 단어의 경우 OOV
        new_x.append(word_to_index['OOV'])

print("새로운 유형의 시퀀스 : ", new_x)


# .. NER 예측
# 예측
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x], padding="post", value=0, maxlen=max_len)
p = model.predict(np.array([new_padded_seqs[0]]))
p = np.argmax(p, axis=-1) # 예측된 NER 인덱스값 추출

# 출력
# 새로운 문장에 대한 BIO 정답이 없어 맞았는지 안맞았는지 모름
print("{:10} {:5}".format("단어", "예측된 NER"))
print("-" * 50)
for w, pred in zip(new_sentence, p[0]):
    print("{:10} {:5}".format(w, index_to_ner[pred]))

c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_PS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_OG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_TI seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Python38\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_LC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

           _       0.65      0.59      0.62       657
         _DT       0.95      0.89      0.92       335
         _LC       0.75      0.53      0.62       312
         _OG       0.76      0.54      0.63       481
         _PS       0.85      0.38      0.52       374
         _TI       0.95      0.80      0.87        66

   micro avg       0.77      0.59      0.67      2225
   macro avg       0.82      0.62      0.70      2225
weighted avg       0.78      0.59      0.66      2225

F1-score: 66.6%
새로운 유형의 시퀀스 :  [531, 307, 1476, 286, 1507, 6766, 1]
1/1 [==============================] - 0s 27ms/step
단어         예측된 NER
--------------------------------------------------
삼성전자       B_OG 
출시         O    
스마트폰       O    
오늘         B_DT 
애플         B_OG 
도전장        I    
내밀다.       I    
